In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import segmentation_models_pytorch as smp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import open3d as o3d
import os
from PIL import Image
from IPython.display import display

# Filter harmless warnings
import warnings
warnings.filterwarnings("ignore")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Testing data

In [3]:
# df = pd.read_csv('./testFiles/data.csv') 
# df_INPUT_DEPTH = df[['depth_img_I', 'depth_img_II']]
# df_INPUT_RGB = df[['rgb_img_I', 'rgb_img_II']]
# df_OUTPUT = df[['x1','y1','z1','x2','y2','z2']]

In [4]:
from sympy import Line2D, Point2D, Segment3D, Point3D, Line3D, Symbol, solve
from sympy.plotting import plot as symplot

from sympy import symbols
from numpy import linspace
from sympy import lambdify
import numpy as np
import cv2 as cv
import pandas as pd
import os
import csv
import time
import tqdm
import threading as thr
from shutil import copyfile
import multiprocessing as mp
# from visualiser import Visualiser
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from PIL import Image as im

def draw_point(pt, img, img_size, divider, K):
    new_K = np.resize(K, (3, 3))
    X = np.matmul(new_K, [pt[0], pt[1], pt[2]])
    x1 = int(X[0] / X[2] / divider)
    y1 = int(X[1] / X[2] / divider)
    if y1 > img_size[0] or x1 > img_size[1] or y1 < 0 or x1 < 0:
        return False
    image = np.zeros(img_size, float)
    image = cv.circle(image, (x1, y1), radius=1, color=255, thickness=-1)
    # image = cv.flip(image, 1)
    img[np.where(image==255)] = pt[2]
    return True

def calc_xz_for_y(pt1, pt2, y):
    t = (y - pt1[1])/(pt2[1] - pt1[1])
    x = pt1[0] + (pt2[0] - pt1[0]) * t
    z = pt1[2] + (pt2[2] - pt1[2]) * t
    # if x> 2 or x < -2:
    #     print(x, y, z)
    # print([x, y, z])
    return [x,y,z]

def get_pixel(pt, img_size, divider, K):
    new_K = np.resize(K, (3, 3))
    X = np.matmul(new_K, [pt[0], pt[1], pt[2]])
    x1 = int(X[0] / X[2] / divider)
    y1 = int(X[1] / X[2] / divider)
    if y1 > img_size[0] or x1 > img_size[1] or y1 < 0 or x1 < 0:
        return
    return x1, y1, pt[2]

def convert_csv(p1a, p2a):
    DIVIDER = 1
    IMG_SIZE = (480//DIVIDER, 640//DIVIDER)

    img = np.zeros(IMG_SIZE, float)

    pt1 = p1a
    pt2 = p2a

    K = [570.0,0.0,320.0,0.0,570.0,240.0,0.0,0.0,1.0]

    switch_to_Y = abs(pt2[0]-pt1[0]) < abs(pt2[1]-pt1[1])

    min_ran = pt1[1]
    max_ran = pt2[1]

    if switch_to_Y:
        step = (max_ran - min_ran) / (IMG_SIZE[1])
    else:
        step = (max_ran - min_ran) / (IMG_SIZE[0])
    ran = np.arange(min_ran, max_ran, step)
    a = []
    if switch_to_Y:
        for y_i in ran:
            a.append(calc_xz_for_y(pt1, pt2, y_i))
    else:
        for x_i in ran:
            a.append(calc_yz_for_x(pt1, pt2, x_i))

    a_px = [get_pixel(px, IMG_SIZE, DIVIDER, K) for px in a]
    a_px = [np.array([int(i[0]), int(i[1]), float(i[2])]) for i in a_px if i is not None]

    linspace = np.linalg.norm(a_px[0] - a_px[-1])
    points_on_line = np.linspace(a_px[0], a_px[-1], int(linspace))
    for pt in points_on_line:
        tmp_img = np.zeros(IMG_SIZE, float)
        # depth_img = np.load(data['depth_img_II'].iloc[i])
        # depth_img = cv.imread(data['depth_img_I'].iloc[i], cv.IMREAD_UNCHANGED)
        cv.circle(tmp_img, (int(pt[0]), int(pt[1])), radius=5, color=pt[2], thickness=-1)
        # if np.sum(tmp_img*depth_img):
        img[np.where(tmp_img!=0)] = tmp_img[np.where(tmp_img!=0)]
        
    return img
        


In [5]:
ResizeData = transforms.Resize([256, 320], interpolation=transforms.InterpolationMode.NEAREST)

# Functions 
# Functions 
def CreatePointCloud(color_im, depth_im):
    color_raw = o3d.geometry.Image(np.uint8(color_im))
    depth_raw = o3d.geometry.Image(np.float32(depth_im))
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, 1000) # 
    PointCloud = o3d.geometry.PointCloud.create_from_rgbd_image(
      rgbd_image,o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)) # Creates Point Cloud from rgbd image
#     PointCloud.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down
    return PointCloud

def CreateAxisCloud(depth_im):
    depth_raw  = o3d.geometry.Image(np.float32(depth_im/1)) # Converts depth data into image format
    PointCloud = o3d.geometry.PointCloud.create_from_depth_image(depth_raw,o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
    # PointCloud.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down
    return PointCloud

def pick_points(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    numpy_array=np.asarray(pcd.points)
    point_id=vis.get_picked_points()

    return [numpy_array[point_id[0]],numpy_array[point_id[1]]]

def draw_arrow(pcd, points_real, points_extimated):
    lines=[[0,1],[2,3]]
    points = np.concatenate((points_real, points_extimated), axis=0)
    colors = [[1,0,0],[0,1,0]] # Red is REAL and Green is ESTIMATED
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),

    )
    line_set.colors=o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd,line_set])

In [6]:
# RGBimg_begin = load_from_csv(df_INPUT_RGB, 'rgb' ,0)
# DEPTHimg_begin = load_from_csv(df_INPUT_DEPTH, 'depth',0)
# DEPTHimg_end = load_from_csv(df_INPUT_DEPTH, 'depth', 1)

# # Taking first rgb image
# rgb_in = torch.IntTensor(RGBimg_begin.transpose(0,3,1,2))
# gray_in = RGBtoGRAY(rgb_in).div(255)
# # plt.imshow(gray_in[0].numpy().transpose(1,2,0),cmap='gray')

# # Taking depth beginning state of the movement
# depthBeg_in = NormImage(DEPTHimg_begin,5500,reshape=True)

# # Taking depth end state of the movement
# depthEnd_in = NormImage(DEPTHimg_end,5500,reshape=True)

# # Taking depth difference between movements
# depthDiff_in = NormImage(abs(DEPTHimg_begin - DEPTHimg_end),5500,reshape=True)

# # Taking outputs
# axis_out = df_OUTPUT.values
# y_test = torch.Tensor(axis_out)

# X_test = ResizeData(torch.cat((gray_in, depthBeg_in, depthDiff_in),axis=1))
# print(RGBD_input.shape)
# # RGBD_input = DDD_in

In [7]:
from Libraries.dataloader import DataLoader as DL

DATASET_ROOTDIR='/home/el_zlociako/Documents/Praca_inzynierska/Dataset/'
dl = DL()

X, y = dl.load(DATASET_ROOTDIR, 'files/data.csv', 'R')
X_test, y_test = dl.load(DATASET_ROOTDIR, 'files_Test/data_Test.csv', 'R')

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=101)

# X_train_aug = X_train.clone()
# for i in range(X_train_aug.shape[0]):
#      X_train_aug[i] = DataAug(X_train_aug[i])
        
# X_train = torch.cat((X_train, X_train_aug),axis=0)
# y_train = torch.cat((y_train, y_train),axis=0)


In [8]:
train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_validation, y_validation)
test_set = TensorDataset(X_test, y_test)

loader_args = dict(batch_size=10, num_workers=os.cpu_count(), pin_memory=True, drop_last=True)
train_loader = DataLoader(train_set,shuffle=True, **loader_args)
val_loader = DataLoader(val_set,shuffle=False, **loader_args)
test_loader = DataLoader(test_set,shuffle=False, **loader_args)

In [9]:
# plt.imshow(np.transpose(depthDiff_in[2].numpy(),(1,2,0)))

In [10]:
# plt.imshow(np.transpose(rgb_in[6].numpy(),(1,2,0)))

In [11]:
class AoRNet(nn.Module):
    def __init__(self,pretrained=False ,input_channels=3, output_size=6):
        super().__init__()
        self.resnet50 = models.resnet50(pretrained=pretrained)
        self.resnet50.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet50.fc = nn.Linear(in_features=2048, out_features=output_size, bias=True)
    
    def forward(self, X):
        return self.resnet50(X)

In [36]:
Model = AoRNet()
Model.load_state_dict(torch.load('Modele/Big2_RN_150Epoch'))
Model.eval()

AoRNet(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [37]:
inv_resize = transforms.Resize(480, interpolation=transforms.InterpolationMode.NEAREST)
LossFCN = nn.MSELoss()

In [40]:
# torch.manual_seed(101)
val_err = []

with torch.no_grad():
    for b, (X_validation, y_validation) in enumerate(val_loader):
#         Apply the model
        y_val = Model(X_validation)
#         print(y_val.shape)
        for j in range(y_val.shape[0]):
            X_invNorm = inv_resize(X_validation[j])
            RGB_buff = X_invNorm[0].numpy()*255
#             RGB_buff = np.stack((X_invNorm[0].numpy(),X_invNorm[1].numpy(),X_invNorm[2].numpy()))*255
#             RGB_buff = np.transpose(RGB_buff, (1,2,0))
            RGB_buff = np.ascontiguousarray(RGB_buff, dtype=np.uint8)
    
#             X1E = y_val[j][0].cpu().numpy()
#             Y1E = y_val[j][1].cpu().numpy()
#             Z1E = y_val[j][2].cpu().numpy()
            
#             X2E = y_val[j][3].cpu().numpy()
#             Y2E = y_val[j][4].cpu().numpy()
#             Z2E = y_val[j][5].cpu().numpy()
            
#             X1R = y_validation[j][0].cpu().numpy()
#             Y1R = y_validation[j][1].cpu().numpy()
#             Z1R = y_validation[j][2].cpu().numpy()
            
#             X2R = y_validation[j][3].cpu().numpy()
#             Y2R = y_validation[j][4].cpu().numpy()
#             Z2R = y_validation[j][5].cpu().numpy()
            
            
#             print(f'----------------------------------------------------------------------------------------------')
#             print(f'REAL:')
#             SEG_REAL = convert_csv([X1R, Y1R, Z1R],  [X2R, Y2R, Z2R])
            
#             print(f'ESTIMATED:')
#             SEG_ESTI = convert_csv([X1E, Y1E, Z1E],  [X2E, Y2E, Z2E])
            
#             tp, fp, fn, tn = smp.metrics.get_stats(torch.Tensor(SEG_ESTI).long(), torch.Tensor(SEG_REAL).long(), mode='binary', threshold=0.1)
#             err = smp.metrics.f1_score(tp, fp, fn, tn)
#             err = LossFCN(torch.Tensor(SEG_ESTI), torch.Tensor(SEG_REAL))
#             val_err.append(err.cpu().detach().numpy())
            
            print(f'----------------------------------------------------------------------------------------------')
            
#             DEPTH_buff = X_invNorm[1].numpy()*5500
            
#             PC_object = CreatePointCloud(RGB_buff, DEPTH_buff)
#             PC_realAxis = CreateAxisCloud(SEG_REAL)
#             PC_predAxis = CreateAxisCloud(SEG_ESTI)
            
#             o3d.visualization.draw_geometries([PC_object,PC_predAxis])

            print(f'--> BATCH: {b+1} <-- | --> ROW: {j} <--')
            print(f'----------------------------------------------------------------------------------------------')
            print(f'{"X1":>12} {"Y1":>12} {"Z1":>12} {"X2":>12} {"Y2":>12} {"Z2":>12}')
            print(f'{"PREDICTED:"}')
            print(f'[[{y_val[j][0]:12.5f}, {y_val[j][1]:12.5f}, {y_val[j][2]:12.5f}], [{y_val[j][3]:12.5f}, {y_val[j][4]:12.5f}, {y_val[j][5]:12.5f}]]')
            print(f'{"REAL:"}')
            print(f'[[{y_validation[j][0]:12.5f}, {y_validation[j][1]:12.5f}, {y_validation[j][2]:12.5f}], [{y_validation[j][3]:12.5f}, {y_validation[j][4]:12.5f}, {y_validation[j][5]:12.5f}]]')
            print(f'{"DIFFERENCE:"}')
            diff = np.abs(y_val.cpu().numpy()-y_validation.cpu().numpy())# 
#             print(diff)
            val_err.append([diff[j][0], diff[j][1], diff[j][2], diff[j][3], diff[j][4], diff[j][5]])
#             print(f'[[{diff[j][0]:12.5f}, {diff[j][1]:12.5f}, {diff[j][2]:12.5f}], [{diff[j][3]:12.5f}, {diff[j][4]:12.5f}, {diff[j][5]:12.5f}]]')())
            print(f'[[{diff[j][0]:12.5f}, {diff[j][1]:12.5f}, {diff[j][2]:12.5f}], [{diff[j][3]:12.5f}, {diff[j][4]:12.5f}, {diff[j][5]:12.5f}]]')
            
            print(f'----------------------------------------------------------------------------------------------')
# loss = criterion(y_val, y_validation.cuda())
# diff = np.abs(y_val.cpu().numpy()-y_validation.cpu().numpy())
# print(f'RMSE: {loss:.8f}')


----------------------------------------------------------------------------------------------
--> BATCH: 1 <-- | --> ROW: 0 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[    -0.13260,     -0.53972,      1.85931], [    -0.13040,      0.62878,      1.79380]]
REAL:
[[    -0.02617,     -0.66037,      2.11400], [    -0.03380,      0.93092,      2.21650]]
DIFFERENCE:
[[     0.10642,      0.12065,      0.25469], [     0.09660,      0.30213,      0.42270]]
----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
--> BATCH: 1 <-- | --> ROW: 1 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDI

----------------------------------------------------------------------------------------------
--> BATCH: 3 <-- | --> ROW: 0 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[    -0.06301,     -0.46432,      1.71153], [    -0.04907,      0.50875,      1.65117]]
REAL:
[[     0.09985,     -0.33658,      1.17800], [     0.10554,      0.18001,      1.30350]]
DIFFERENCE:
[[     0.16287,      0.12774,      0.53353], [     0.15461,      0.32874,      0.34767]]
----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
--> BATCH: 3 <-- | --> ROW: 1 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDI

In [41]:
for i in range(6):
    print(f'MEAN XYZ{i}: {np.mean(val_err[i])}')
    print(f' STD XYZ{i}: {np.std(val_err[i])}')

MEAN XYZ0: 0.2172006368637085
 STD XYZ0: 0.12040547281503677
MEAN XYZ1: 0.16703279316425323
 STD XYZ1: 0.08438583463430405
MEAN XYZ2: 0.19939564168453217
 STD XYZ2: 0.08229247480630875
MEAN XYZ3: 0.27692553400993347
 STD XYZ3: 0.12916994094848633
MEAN XYZ4: 0.5221603512763977
 STD XYZ4: 0.48801806569099426
MEAN XYZ5: 0.16588915884494781
 STD XYZ5: 0.10219450294971466


In [28]:
np.std(val_err[0])

0.19688192

In [93]:
# aaa
Model.eval()

num = 3 #6 8 11 19 30 #1 8 12 23 31
One_photo_x = X_validation[num].reshape(1,
                                      X_validation[num].shape[0],
                                      X_validation[num].shape[1],
                                      X_validation[num].shape[2]
                                     )

One_photo_y = y_validation[num]

# One_photo_x = X_test[num].reshape(1,
#                                       X_test[num].shape[0],
#                                       X_test[num].shape[1],
#                                       X_test[num].shape[2]
#                                      )

# One_photo_y = y_test[num].reshape(1,
#                                       y_test[num].shape[0],
#                                       y_test[num].shape[1],
#                                       y_test[num].shape[2]
#                                      ).to(device)

with torch.no_grad():
    y_val = Model(One_photo_x)

    images_buff = inv_resize(One_photo_x[0])
    RGB_buff = images_buff[0].cpu().numpy()*255
    RGB_buff = np.ascontiguousarray(RGB_buff, dtype=np.uint8)

    X1E = y_val[0][0].cpu().numpy()
    Y1E = y_val[0][1].cpu().numpy()
    Z1E = y_val[0][2].cpu().numpy()

    X2E = y_val[0][3].cpu().numpy()
    Y2E = y_val[0][4].cpu().numpy()
    Z2E = y_val[0][5].cpu().numpy()

    X1R = One_photo_y[0].cpu().numpy()
    Y1R = One_photo_y[1].cpu().numpy()
    Z1R = One_photo_y[2].cpu().numpy()

    X2R = One_photo_y[3].cpu().numpy()
    Y2R = One_photo_y[4].cpu().numpy()
    Z2R = One_photo_y[5].cpu().numpy()

    SEG_REAL = convert_csv([X1R, Y1R, Z1R],  [X2R, Y2R, Z2R])
    SEG_ESTI = convert_csv([X1E, Y1E, Z1E],  [X2E, Y2E, Z2E])
    
    DEPTH_buff = images_buff[1].numpy()*5500
    
    PC_object = CreatePointCloud(RGB_buff, DEPTH_buff)
    PC_realAxis = CreateAxisCloud(SEG_REAL)
    PC_predAxis = CreateAxisCloud(SEG_ESTI)

    o3d.visualization.draw_geometries([PC_object, PC_realAxis])
    o3d.visualization.draw_geometries([PC_object, PC_predAxis])
